# Reviewer Randomizer

**Inputs:**
1. Number of people to be assigned to each applicant.
2. List of reviewer names.
3. List of applicant names.

**Outputs:**
1. Table of applicants with randomly assigned reviewers.
2. Table of reviewers with assigned applicants to review.

*(Both output tables contain the same information in different format.)*

In [1]:
# Load packages
from random import randrange
from math import ceil
from google.colab import files

### Enter inputs:
* **Number of eyes on each application** (`eyes`)
* **Reviewer names** (`reviewers`) - each reviewer on a new line; 3 quotes (`"""`) at the beginning and end of the list. Names can be first and last or just first or last; first and last names separated by space or a tab (doesn't matter). You should be able to paste the list in from excel if desired.
* **Applicant names** (`applicants`) - same layout as reviewers

In [2]:
# Number of people assigned to each applicant.
eyes = 3

In [3]:
# Input list of reviewers. Need 3 quotation marks at beginning and end of list only. Each reviewer should be on a new line.
reviewers = """Gary
Tianyi
Kevin
Larry
Henrique
Eric
Kelly
Haining
John"""
reviewers = reviewers.replace("\t", " ").split("\n")

In [4]:
# Input list of applicants. Need 3 quotation marks at beginning and end of list only. Each reviewer should be on a new line.
applicants = """A	B
C	D
E	F
G	H
I	J
K	L
M	N
O	P
Q	R
S	T
U	V
W	X
Y	Z"""
applicants = applicants.replace("\t", " ").split("\n")

### Run Script, Export Excel Sheets

In [5]:
reviewer_num = len(reviewers)
applicant_num = len(applicants)
max_num = ceil(applicant_num * eyes / reviewer_num)

reviewer_counts = {} # keep track of how many students each reviewer is assigned
reviewer_dict = {} # keep a list of each applicant assigned to each reviewer
applicant_dict = {} # keep a list of each reviewer assigned to each applicant
for reviewer in reviewers: # build reviewer dictionaries
    reviewer_counts[reviewer] = 0
    reviewer_dict[reviewer] = []

# save table of applicant: reviewers
with open("applicants_reviewers.csv","w") as output:
  output.write("Applicant,Reviewers" + ","*(eyes-1) + "\n")
  print("Applicant\tReviewers")
  for applicant in applicants:
      applicant_dict[applicant] = []
      rev_list = []
      line = applicant
      for x in range(eyes):
          temp = reviewers[randrange(reviewer_num)]
          while temp in rev_list or reviewer_counts[temp] >= max_num:
              temp = reviewers[randrange(reviewer_num)]
          rev_list.append(temp)
          reviewer_counts[temp] += 1
          reviewer_dict[temp].append(applicant)
          applicant_dict[applicant].append(temp)
          line += "," + temp
      output.write(line + "\n")
      print(line.replace(",","\t"))
files.download("applicants_reviewers.csv")

Applicant	Reviewers
A B	Gary	Tianyi	Kevin
C D	Henrique	Eric	Kelly
E F	John	Gary	Tianyi
G H	Henrique	John	Haining
I J	Haining	Kevin	Gary
K L	Eric	Larry	John
M N	Eric	Tianyi	Larry
O P	Kevin	Eric	Haining
Q R	Eric	Tianyi	Larry
S T	Kelly	John	Kevin
U V	Gary	Tianyi	John
W X	Larry	Henrique	Haining
Y Z	Haining	Henrique	Gary


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# check distribution of work
for reviewer, count in reviewer_counts.items():
    print(f"{reviewer}\t{count}")

Gary	5
Tianyi	5
Kevin	4
Larry	4
Henrique	4
Eric	5
Kelly	2
Haining	5
John	5


In [7]:
# save table of reviewer: applicants
with open("reviewer_applicants.csv","w") as output:
  output.write("Reviewer,Applicants" + ","*(max_num-1) + "\n")
  print("Reviewer\tApplicants")
  for reviewer in reviewer_dict:
      line = reviewer
      for applicant in reviewer_dict[reviewer]:
        line += "," + applicant
      while line.count(",") < max_num:
        line += ","
      output.write(line + "\n")
      print(line.replace(",","\t"))
files.download("reviewer_applicants.csv")

Reviewer	Applicants
Gary	A B	E F	I J	U V	Y Z
Tianyi	A B	E F	M N	Q R	U V
Kevin	A B	I J	O P	S T	
Larry	K L	M N	Q R	W X	
Henrique	C D	G H	W X	Y Z	
Eric	C D	K L	M N	O P	Q R
Kelly	C D	S T			
Haining	G H	I J	O P	W X	Y Z
John	E F	G H	K L	S T	U V


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>